In [1]:
# library
import torch
import numpy as np
import pandas as pd
from torchmetrics import R2Score, MeanSquaredError

r2score   = R2Score()
msescore  = MeanSquaredError()

torch.manual_seed(1)
np.random.seed(1)

In [2]:
# Model
class Net(torch.nn.Module):
    def __init__(self, input_size=1, output_size=1, depth=1, width=4):
        super(Net, self).__init__()
        self.input   = torch.nn.Linear(input_size, width)
        self.linears = torch.nn.ModuleList([torch.nn.Linear(width, width) for i in range(depth-1)])
        self.output  = torch.nn.Linear(width, output_size)

    def forward(self, x):
        z = torch.selu(self.input(x))
        for linear in(self.linears):
            z = torch.selu(linear(z))
        z = torch.relu(self.output(z))
        return z

In [3]:
# Model evaluation
def eval(model, testset):
    with torch.no_grad():
        pred_Y = model(testset.x_data)
    
    r2  = r2score(pred_Y, testset.y_data)
    mse = msescore(pred_Y, testset.y_data)
    return r2.item(), mse.item()

In [4]:
# Data class
class Data(torch.utils.data.Dataset):
  def __init__(self, src_file, start=None, end=None):
    df = pd.read_csv(src_file)
    Tz_k   = np.array(df['room_temp']).reshape(-1,1)[start: end]
    # To_k   = np.array(df['thermostat_outside_temp']).reshape(-1,1)[start: end]
    Tsa_k1 = np.array(df['supply_discharge_temp']).reshape(-1,1)[start+1: end+1]
    msa_k1 = np.array(df['airflow_current']).reshape(-1,1)[start+1: end+1]

    # tmp_x = np.concatenate((Tz_k, Tsa_k1, msa_k1, To_k), axis=1)
    tmp_x = np.concatenate((Tz_k, Tsa_k1, msa_k1), axis=1)
    tmp_y = np.array(df['room_temp']).reshape(-1,1)[start+1: end+1]

    self.x_data = torch.tensor(tmp_x, dtype=torch.float32)
    self.y_data = torch.tensor(tmp_y, dtype=torch.float32)

  def __len__(self):
    return len(self.x_data)

  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()
    inp  = self.x_data[idx]
    outp = self.y_data[idx]
    sample = {'inp':inp, 'outp':outp}
    return sample

In [5]:
# Early stopping function
def early_stop(list, min_epochs, patience):
    if(len(list) > min_epochs):
        if(np.max(list[-patience:]) < 1.00001*np.max(list[0: -patience])):
            return 1
    return 0

In [6]:
# train function
def train(net, train_ds, test_ds, lr=0.001, min_epochs=200, max_epochs=100000, patience=100, smooth=0):
    loss_func  = torch.nn.MSELoss()
    optimizer  = torch.optim.Adam(net.parameters(), lr=lr)

    R2_test = np.array([])
    MSE_test = np.array([])
    train_ldr = torch.utils.data.DataLoader(train_ds, batch_size=train_ds.y_data.shape[0], shuffle=True)
    for _ in range(0, max_epochs+1):
        net.train()
        for (_, batch) in enumerate(train_ldr):
            X = batch['inp']
            Y = batch['outp']

            optimizer.zero_grad()
            output = net(X)
            loss_val = loss_func(output, Y) + smooth*loss_func(output, X[:,0].reshape(-1,1))
            loss_val.backward()
            optimizer.step()

        net.eval()
        R2_test  = np.append(R2_test, eval(net, test_ds)[0])
        MSE_test = np.append(MSE_test, eval(net, test_ds)[1])
        
        if(early_stop(list = R2_test, min_epochs = min_epochs, patience = patience) == 1):
            break
    
    return R2_test, MSE_test

In [7]:
# main
df_result = pd.DataFrame({'n_train':[], 'smooth':[], 'depth':[], 'width':[], 'lr':[], 'best_epoch':[], 'R2':[], 'MSE':[]})
n_end = 1600

for _n_train in [16, 32, 64, 128]:
    for _smooth in [0, 0.001, 0.01]:
        for _depth in [1, 2, 3, 4]:
            for _width in [3, 4, 16, 32]:
                for _lr in [0.0001, 0.001, 0.01]:

                    # Create network
                    device = torch.device("cpu")
                    net = Net(input_size=3, output_size=1, depth=_depth, width=_width).to(device)

                    # Create Dataset and DataLoader objects
                    src_file = 'C:/Users/tln229/Downloads/Code/Python PIML/1. Building/data/HVAC_B90_102_exp_10m_20210424.csv'
                    n_train  = _n_train
                    train_ds = Data(src_file, start=0,       end=n_train)
                    test_ds  = Data(src_file, start=n_train, end=n_end)

                    # train
                    R2_test, MSE_test = train(net, train_ds, test_ds, lr=_lr, min_epochs=500, max_epochs=100000, patience=200, smooth=_smooth)

                    # results
                    df_result.loc[len(df_result)] = [_n_train, _smooth, _depth, _width, _lr, np.argmax(R2_test), np.max(R2_test), MSE_test[np.argmax(R2_test)]]

In [8]:
df_result.to_csv("C:/Users/tln229/Downloads/Code/Python PIML/1. Building/Finals/1. Zone model/NN_smooth_stat.csv")

In [9]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 4,
                       ):
    print(df_result)

     n_train  smooth  depth  width      lr  best_epoch          R2         MSE
0       16.0   0.000    1.0    3.0  0.0001         0.0  -1524.0552   5158.1333
1       16.0   0.000    1.0    3.0  0.0010         2.0  -4753.8721  16082.2148
2       16.0   0.000    1.0    3.0  0.0100         0.0  -1524.0552   5158.1333
3       16.0   0.000    1.0    4.0  0.0001         0.0  -1524.0552   5158.1333
4       16.0   0.000    1.0    4.0  0.0010         2.0  -4116.5078  13926.4824
5       16.0   0.000    1.0    4.0  0.0100         0.0  -1524.0552   5158.1333
6       16.0   0.000    1.0   16.0  0.0001         0.0   -444.9655   1508.3713
7       16.0   0.000    1.0   16.0  0.0010         0.0  -1524.0552   5158.1333
8       16.0   0.000    1.0   16.0  0.0100         0.0   -808.2196   2736.9912
9       16.0   0.000    1.0   32.0  0.0001         0.0   -514.2201   1742.6084
10      16.0   0.000    1.0   32.0  0.0010         0.0  -1524.0552   5158.1333
11      16.0   0.000    1.0   32.0  0.0100         0